1. Importing
2. Basic Exploration
3. Dropping Irrelevant columns
4. Checking for null values
5. Checking constraints for age and capping avg watch time per day as per no. of profiles

In [4]:
import pandas as pd
import numpy as np

In [5]:
df=pd.read_csv('../data/raw/netflix_customer_churn.csv')


In [6]:
df.head()

,customer_id,age,gender,subscription_type,watch_hours,last_login_days,region,device,monthly_fee,churned,payment_method,number_of_profiles,avg_watch_time_per_day,favorite_genre
0,a9b75100-82a8-427a-a208-72f24052884a,51,Other,Basic,14.73,29,Africa,TV,8.99,1,Gift Card,1,0.49,Action
1,49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b,47,Other,Standard,0.70,19,Europe,Mobile,13.99,1,Gift Card,5,0.03,Sci-Fi
2,4d71f6ce-fca9-4ff7-8afa-197ac24de14b,27,Female,Standard,16.32,10,Asia,TV,13.99,0,Crypto,2,1.48,Drama
3,d3c72c38-631b-4f9e-8a0e-de103cad1a7d,53,Other,Premium,4.51,12,Oceania,TV,17.99,1,Crypto,2,0.35,Horror
4,4e265c34-103a-4dbb-9553-76c9aa47e946,56,Other,Standard,1.89,13,Africa,Mobile,13.99,1,Crypto,2,0.13,Action


In [7]:
df.columns

Index(['customer_id', 'age', 'gender', 'subscription_type', 'watch_hours',
       'last_login_days', 'region', 'device', 'monthly_fee', 'churned',
       'payment_method', 'number_of_profiles', 'avg_watch_time_per_day',
       'favorite_genre'],
      dtype='object')

In [8]:
mapping = {"Basic": 8.99, "Standard": 13.99, "Premium": 17.99}
df["expected_fee"] = df["subscription_type"].map(mapping)

# Check mismatches
mismatches = df[df["expected_fee"] != df["monthly_fee"]]
print(mismatches.shape)


(0, 15)


In [9]:
# Drop customer_id
df.drop(columns=['customer_id','monthly_fee','expected_fee'], inplace=True)



In [10]:
df.info()
print(df.isnull().sum())
print(df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   age                     5000 non-null   int64  
 1   gender                  5000 non-null   object 
 2   subscription_type       5000 non-null   object 
 3   watch_hours             5000 non-null   float64
 4   last_login_days         5000 non-null   int64  
 5   region                  5000 non-null   object 
 6   device                  5000 non-null   object 
 7   churned                 5000 non-null   int64  
 8   payment_method          5000 non-null   object 
 9   number_of_profiles      5000 non-null   int64  
 10  avg_watch_time_per_day  5000 non-null   float64
 11  favorite_genre          5000 non-null   object 
dtypes: float64(2), int64(4), object(6)
memory usage: 468.9+ KB
age                       0
gender                    0
subscription_type         0
wat

In [11]:
df.describe()

,age,watch_hours,last_login_days,churned,number_of_profiles,avg_watch_time_per_day
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,43.847400,11.649450,30.089800,0.503000,3.024400,0.874800
std,15.501128,12.014654,17.536078,0.500041,1.415841,2.619824
min,18.000000,0.010000,0.000000,0.000000,1.000000,0.000000
25%,30.000000,3.337500,15.000000,0.000000,2.000000,0.110000
50%,44.000000,8.000000,30.000000,1.000000,3.000000,0.290000
75%,58.000000,16.030000,45.000000,1.000000,4.000000,0.720000
max,70.000000,110.400000,60.000000,1.000000,5.000000,98.420000


In [12]:
df.describe(include='object')

,gender,subscription_type,region,device,payment_method,favorite_genre
count,5000,5000,5000,5000,5000,5000
unique,3,3,6,5,5,7
top,Female,Premium,South America,Tablet,Debit Card,Drama
freq,1711,1693,873,1048,1030,731


In [13]:
# Function to cap avg_watch_time_per_day based on profiles
def cap_watch_time(row):
    max_allowed = row['number_of_profiles'] * 24
    return min(row['avg_watch_time_per_day'], max_allowed)

df['avg_watch_time_per_day'] = df.apply(cap_watch_time, axis=1)


In [14]:
df.groupby('number_of_profiles')['avg_watch_time_per_day'].max()


number_of_profiles
1    24.00
2    48.00
3    32.81
4    34.53
5    98.42
Name: avg_watch_time_per_day, dtype: float64

In [15]:
df['avg_watch_time_per_day'].max()

98.42

In [16]:
df["age"].max()

70

In [17]:
df[(df["age"] >= 18) & (df["age"] <= 70)]


,age,gender,subscription_type,watch_hours,last_login_days,region,device,churned,payment_method,number_of_profiles,avg_watch_time_per_day,favorite_genre
0,51,Other,Basic,14.73,29,Africa,TV,1,Gift Card,1,0.49,Action
1,47,Other,Standard,0.70,19,Europe,Mobile,1,Gift Card,5,0.03,Sci-Fi
2,27,Female,Standard,16.32,10,Asia,TV,0,Crypto,2,1.48,Drama
3,53,Other,Premium,4.51,12,Oceania,TV,1,Crypto,2,0.35,Horror
4,56,Other,Standard,1.89,13,Africa,Mobile,1,Crypto,2,0.13,Action
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,19,Female,Basic,49.17,11,Europe,Desktop,0,Credit Card,4,4.10,Drama
4996,67,Female,Basic,9.24,2,North America,Desktop,0,PayPal,3,3.08,Documentary
4997,66,Male,Standard,16.55,49,South America,Desktop,1,Debit Card,2,0.33,Action
4998,59,Female,Basic,9.12,3,Europe,Laptop,0,Credit Card,4,2.28,Sci-Fi


In [18]:
df

,age,gender,subscription_type,watch_hours,last_login_days,region,device,churned,payment_method,number_of_profiles,avg_watch_time_per_day,favorite_genre
0,51,Other,Basic,14.73,29,Africa,TV,1,Gift Card,1,0.49,Action
1,47,Other,Standard,0.70,19,Europe,Mobile,1,Gift Card,5,0.03,Sci-Fi
2,27,Female,Standard,16.32,10,Asia,TV,0,Crypto,2,1.48,Drama
3,53,Other,Premium,4.51,12,Oceania,TV,1,Crypto,2,0.35,Horror
4,56,Other,Standard,1.89,13,Africa,Mobile,1,Crypto,2,0.13,Action
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,19,Female,Basic,49.17,11,Europe,Desktop,0,Credit Card,4,4.10,Drama
4996,67,Female,Basic,9.24,2,North America,Desktop,0,PayPal,3,3.08,Documentary
4997,66,Male,Standard,16.55,49,South America,Desktop,1,Debit Card,2,0.33,Action
4998,59,Female,Basic,9.12,3,Europe,Laptop,0,Credit Card,4,2.28,Sci-Fi


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   age                     5000 non-null   int64  
 1   gender                  5000 non-null   object 
 2   subscription_type       5000 non-null   object 
 3   watch_hours             5000 non-null   float64
 4   last_login_days         5000 non-null   int64  
 5   region                  5000 non-null   object 
 6   device                  5000 non-null   object 
 7   churned                 5000 non-null   int64  
 8   payment_method          5000 non-null   object 
 9   number_of_profiles      5000 non-null   int64  
 10  avg_watch_time_per_day  5000 non-null   float64
 11  favorite_genre          5000 non-null   object 
dtypes: float64(2), int64(4), object(6)
memory usage: 468.9+ KB


In [20]:
df.describe()

,age,watch_hours,last_login_days,churned,number_of_profiles,avg_watch_time_per_day
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,43.847400,11.649450,30.089800,0.503000,3.024400,0.873610
std,15.501128,12.014654,17.536078,0.500041,1.415841,2.599513
min,18.000000,0.010000,0.000000,0.000000,1.000000,0.000000
25%,30.000000,3.337500,15.000000,0.000000,2.000000,0.110000
50%,44.000000,8.000000,30.000000,1.000000,3.000000,0.290000
75%,58.000000,16.030000,45.000000,1.000000,4.000000,0.720000
max,70.000000,110.400000,60.000000,1.000000,5.000000,98.420000


In [21]:
df[df["watch_hours"]==df["watch_hours"].max()]

,age,gender,subscription_type,watch_hours,last_login_days,region,device,churned,payment_method,number_of_profiles,avg_watch_time_per_day,favorite_genre
3987,70,Male,Basic,110.4,33,Africa,Desktop,0,PayPal,5,3.25,Horror


In [22]:
df.to_csv("cleaned_netflix_customer_churn.csv", index=False)
